In [1]:
import github
from secrets import MY_TOKEN

g = github.Github(login_or_token=MY_TOKEN, per_page=100)

In [2]:
src_file = "selected3.txt"

In [3]:
with open(src_file, 'r') as f:
    repo_full_names = f.readlines()

repo_full_names = list(map(lambda x: x.strip(), repo_full_names))
repo_full_names = list(dict.fromkeys(repo_full_names))
repo_full_names

['misatos/Markus',
 'hbons/SparkleShare',
 'OpenRA/OpenRA',
 'StartupAPI/users',
 'parabuzzle/lifehelpr',
 'aristidb/aws',
 'guard/guard-livereload',
 'macterra/galton',
 'iamnewton/eanewton.com',
 'Kroc/NoNonsenseForum',
 'marklogic-community/tunage',
 'Ohm-panel/core',
 'Pylons/pyramid',
 'drawpile/Drawpile',
 'remotestorage/remotestorage.js',
 'php-coder/mystamps',
 'jmather/AJAXLogger',
 'netty/netty',
 'jmather/majaxDoctrineMediaPlugin',
 'ghantoos/lshell',
 'ogrodnek/csv-serde',
 'erniep888/ScrumTime',
 'praekeltfoundation/vumi',
 'solutionexchange/DSaaS',
 'elastic/logstash',
 'jenkinsci/jenkins',
 'mysociety/alaveteli',
 'aces/Loris',
 'bozuko/bozuko',
 'rstudio/rstudio',
 'abraham/github-anywhere',
 'bowler-framework/Bowler',
 'luiz/vraptor-html-dsl',
 'dlundquist/sniproxy',
 'jmakeig/socrates',
 'klauern/jna_standby_test',
 'davout/bitcoin-central',
 'highlightjs/highlight.js',
 'perkeep/perkeep',
 'rapila/cms-base',
 'exponentcms/exponent-cms',
 'mobile-shell/mosh',
 'phpsec

In [4]:
len(repo_full_names)

46

In [5]:
def is_s_label(label_name):
    return 'security' in label_name.lower()

In [6]:
def prepare_to_csv(text):
    return '"' + text.replace('"', '""') + '"' if text else ""

In [7]:
with open("data/security_issues.csv", 'a',  encoding='utf-8') as f:

    for name in repo_full_names:
        repo = g.get_repo(name)
        sec_issues = set()
        for label in repo.get_labels():
            if is_s_label(label.name):
                sec_issues.update([issue for issue in repo.get_issues(labels=[label])])

        for issue in sec_issues:
            f.write(repo.full_name + ',')
            f.write(str(issue.id) + ',')
            f.write(prepare_to_csv(issue.title) + ',')
            f.write(prepare_to_csv(issue.body) + ',')

            comments = issue.get_comments()
            comments_str = ""
            for c in comments:
                comments_str += c.body
            f.write(prepare_to_csv(comments_str) + ',')
            
            if issue.pull_request:
                f.write('yes,')
            else:
                f.write('no,')
            
            labels = ""
            for label in issue.labels:
                labels += label.name + ','

            f.write(prepare_to_csv(labels))
            
            f.write('\n')


In [8]:
import pandas as pd

data = pd.read_csv("data/security_issues.csv")
data

,repository,number,title,description,comments,is_pr,labels
0,matomo-org/matomo,124434256,Preventing timing attacks on authentication,### Background\r\n\r\nSee http://codahale.com/...,"Having thought about this issue a bit more, **...",no,"c: Security,"
1,matomo-org/matomo,104426717,do not allow to set access to non existing web...,"As a Super User, currently I can set user acce...",NaN,no,"c: Security,"
2,matomo-org/matomo,452436369,rate limit scheduled email reports,Email reports in Matomo can be abused to send ...,Maybe an even better (even though complexer to...,no,"Major,c: Security,"
3,matomo-org/matomo,1207536963,`POST` to `matomo.php` Endpoint Returns 204 Wh...,According to the documentation [in the Matomo ...,"@Zozman thanks for the bug report, I think tha...",no,"c: Security,Regression,"
4,matomo-org/matomo,333223420,Encourage strong passwords by indicating when ...,I think it'd be helpful for the admin to have ...,That could also be added in admin when changin...,no,"Enhancement,c: Security,c: Usability,"
...,...,...,...,...,...,...,...
1227,chrisdickinson/ormnomnom,714830957,[Security] Bump acorn from 7.1.0 to 7.4.1,Bumps [acorn](https://github.com/acornjs/acorn...,NaN,yes,"dependencies,security,"
1228,chrisdickinson/ormnomnom,843557251,[Security] Bump y18n from 3.2.1 to 3.2.2,Bumps [y18n](https://github.com/yargs/y18n) fr...,NaN,yes,"dependencies,security,"
1229,chrisdickinson/ormnomnom,879500828,[Security] Bump hosted-git-info from 2.5.0 to ...,Bumps [hosted-git-info](https://github.com/npm...,NaN,yes,"dependencies,security,"
1230,chrisdickinson/ormnomnom,813492818,[Security] Bump lodash from 4.17.15 to 4.17.21,Bumps [lodash](https://github.com/lodash/lodas...,NaN,yes,"dependencies,security,"


In [9]:
data.title.to_csv("titles.csv")